In [ ]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 8.2 MB/s 
     |████████████████████████████████| 511.7 MB 6.3 kB/s 
     |████████████████████████████████| 1.6 MB 39.2 MB/s 
     |████████████████████████████████| 438 kB 9.4 MB/s 
     |████████████████████████████████| 5.8 MB 21.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninsta

In [ ]:
# reference: https://www.section.io/engineering-education/classification-model-using-bert-and-tensorflow/
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import numpy as np
import ast

In [ ]:
import json

f = open('./database/disease_target.json')
dis = json.load(f)
f.close()

In [ ]:
# get face acupoints
face = []
face_alt = []
for data in dis["data"]:
  
  if data['type'] == 'acupoints':
    if data['attributes']['area'] == 'head':
      if data['attributes']['name'] not in face:
        face.append(data['attributes']['name'])
        for alt in data['attributes']['alt_name']:
          if alt not in face_alt:
            face_alt.append(alt)

print(face)
print(face_alt)

face_alt.remove('光明')
print(face_alt)

['絲竹空', '印堂', '魚腰', '球後', '上迎香', '俠承漿', '睛明', '攢竹', '瞳子髎', '陽白', '承泣', '四白', '巨髎', '地倉', '顴髎', '水溝', '禾髎', '迎香', '承漿']
['目髎', '曲眉', '光明', '眉中', '鼻通', '穿鼻', '下地倉', '痛關', '頦髎', '精明', '淚孔', '淚空', '目內眥', '內眥外', '眉頭', '眉本', '明光', '始光', '夜光', '員柱', '太陽', '前關', '後曲', '魚尾', '客主人', '胃維', '權髎', '兌骨', '人中', '口禾髎', '衝陽', '天池', '懸漿']
['目髎', '曲眉', '眉中', '鼻通', '穿鼻', '下地倉', '痛關', '頦髎', '精明', '淚孔', '淚空', '目內眥', '內眥外', '眉頭', '眉本', '明光', '始光', '夜光', '員柱', '太陽', '前關', '後曲', '魚尾', '客主人', '胃維', '權髎', '兌骨', '人中', '口禾髎', '衝陽', '天池', '懸漿']


In [ ]:
df = pd.read_csv("./database/disease_to_acu_and_symptom_ZH.csv")
df.head(5)

,disease_ID,disease_name,ICD9,meridian,symptoms,acupoints
0,DIS0688,乙型肝炎,NaN,足太陽膀胱經,"['發燒', '食欲減退', '惡心', '嘔吐', '腹痛', '眼白變黃', '小便深色...","['肝俞', '脾俞', '腎俞', '足三里']"
1,SYS0130,口乾,NaN,足太陽膀胱經,[],"['肝俞', '水通穴', '水金穴', '合谷', '期門', '行間', '太沖', '..."
2,DIS0699,子宮內膜異位疼痛,NaN,足太陽膀胱經,[],"['肝俞', '脾俞', '腎俞', '足三里', '合谷', '期門', '太沖', '中..."
3,DIS0272,小兒胃腸病,564,足太陽膀胱經,[],"['肝俞', '脾俞']"
4,DIS0062,小兒營養性消瘦,261,足太陽膀胱經,[],"['肝俞', '胃俞', '身柱', '膈俞', '八風']"


In [ ]:
df['symptoms'] = df['symptoms'].apply(lambda x: ast.literal_eval(x))
df['acupoints'] = df['acupoints'].apply(lambda x: ast.literal_eval(x))
print(type(df.loc[0,'symptoms']), type(df.loc[0,'acupoints']))
df.head(5)

<class 'list'> <class 'list'>


,disease_ID,disease_name,ICD9,meridian,symptoms,acupoints
0,DIS0688,乙型肝炎,NaN,足太陽膀胱經,"[發燒, 食欲減退, 惡心, 嘔吐, 腹痛, 眼白變黃, 小便深色, 大便呈泥土色或變淺色]","[肝俞, 脾俞, 腎俞, 足三里]"
1,SYS0130,口乾,NaN,足太陽膀胱經,[],"[肝俞, 水通穴, 水金穴, 合谷, 期門, 行間, 太沖, 內關, 承漿, 三間, 肺俞,..."
2,DIS0699,子宮內膜異位疼痛,NaN,足太陽膀胱經,[],"[肝俞, 脾俞, 腎俞, 足三里, 合谷, 期門, 太沖, 中脘, 血海, 關元, 三陰交,..."
3,DIS0272,小兒胃腸病,564,足太陽膀胱經,[],"[肝俞, 脾俞]"
4,DIS0062,小兒營養性消瘦,261,足太陽膀胱經,[],"[肝俞, 胃俞, 身柱, 膈俞, 八風]"


In [ ]:
df['disease_name'].value_counts()

乙型肝炎     1
流眼淚      1
脊椎骨痛     1
骨刺       1
腿冷痛      1
        ..
慢性腸炎     1
頭昏眼花     1
大腿痛      1
小腿痛      1
淋巴腺腫瘤    1
Name: disease_name, Length: 840, dtype: int64

In [ ]:
len(df.loc[1,'symptoms'])

0

In [ ]:
# 去除沒有症狀的疾病
for index, row in df.iterrows():
    # check if this disease has no symptom
    if len(df.loc[index,'symptoms']) == 0:
        df = df.drop(index)

print('Row count is:',df.shape[0])

Row count is: 516


In [ ]:
# 去除臉部以外的資料
df_ = pd.DataFrame()
alt = 0
alt_l = []
for index, row in df.iterrows():
    acs = df.loc[index,'acupoints']
    acs_ = []
    for ac in acs:
      if ac in face:
        acs_.append(ac)
      if ac in face_alt:
        alt=alt+1
        acs_.append(ac)
        if ac not in alt_l:
          alt_l.append(ac)

    if acs:
      df.loc[index,'acupoints'] = acs_
      df_ = pd.concat([df.loc[[index]], df_], ignore_index = True, axis = 0)

alt_l

['太陽', '魚尾']

In [ ]:
print('Row count is:',df.shape[0])
print('Row count is:',df_.shape[0])

Row count is: 516
Row count is: 106


In [ ]:
df_.to_csv (r'./database/disease_to_acu_and_symptom_ZH_faceonly.csv', index = False, header=True)

# BERT feature extract

In [ ]:
!pip install -U sentence-transformers 
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 4.4 MB/s 
     |████████████████████████████████| 4.2 MB 18.2 MB/s 
     |████████████████████████████████| 1.2 MB 42.5 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 6.6 MB 44.0 MB/s 
     |████████████████████████████████| 596 kB 47.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=cacb09936e71b6704b7654604d62c17e59677d1cf2f97c8e27015f222b952983
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [ ]:
from sentence_transformers import SentenceTransformer
import scipy.spatial
from tqdm import tqdm
import ast
import pandas as pd
import numpy as np
# load existing data 
try:
  df = pd.read_csv("./database/disease_to_acu_and_symptom_modify_ZH_faceonly_embedding.csv")
  df['symptoms'] = df['symptoms'].apply(lambda x: ast.literal_eval(x))
  df['acupoints'] = df['acupoints'].apply(lambda x: ast.literal_eval(x))
  # df['symptoms_embeddings'] = df['symptoms_embeddings'].apply(lambda x: ast.literal_eval(x))
  print(type(df.loc[0,'symptoms']), type(df.loc[0,'acupoints']),type(df.loc[0,'symptoms_embeddings']),type(df.loc[0,'symptoms_embeddings'][0]))
  print(df.head(5))
  embedder = SentenceTransformer('bert-base-chinese')
except:
  print('no existing data ')

<class 'list'> <class 'list'> <class 'str'> <class 'str'>
  disease_ID disease_name   ICD9      meridian  \
0    DIS0155         眼球震顫  379.5        足太陽膀胱經   
1    DIS0417        新生兒窒息    768            督脈   
2    DIS0206        急性鼻竇炎  461.9        手陽明大腸經   
3    DIS0128         假性近視    367  董氏奇穴七七部為小腿部位   
4    DIS0419        新生兒痙攣    779            督脈   

                                     symptoms_origin  \
0                                           ['眼球震顫']   
1  ['皮膚青紫', '呼吸減慢', '心率減慢', '四肢略屈曲', '四肢癱軟', '對刺激...   
2  ['發燒、流鼻水、咳嗽、頭痛、咽痛，癥狀一直持續下去，鼻水變成化膿性的黃鼻涕，夜間咳嗽變得比...   
3                                         ['看遠看不清楚']   
4  ['點頭單個發作時間短，發作一次為2～10秒，不易察覺；', '全身尤其是頭部和上半身向前屈...   

                                symptoms      acupoints  \
0                                 [眼球震顫]       [睛明, 球後]   
1  [皮膚青紫, 呼吸減慢, 心率減慢, 四肢略屈曲, 四肢癱軟, 反應遲鈍]           [水溝]   
2   [發燒, 流鼻水, 咳嗽, 頭痛, 咽痛, 膿鼻涕, 夜間咳嗽, 臉痛]  [迎香, 印堂, 上迎香]   
3                           [看遠看不清楚, 近視]       [睛明, 承泣]   
4        

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# create symptoms _embeddings
embedder = SentenceTransformer('bert-base-chinese')

df['symptoms_embeddings'] = [[None]] * len(df)
for index, row in tqdm(df.iterrows()):
  df.loc[index,'symptoms_embeddings'] = embedder.encode(df.loc[index,'symptoms'])

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
106it [00:23,  4.59it/s]


In [ ]:
df1 = df.copy()
# change np array to list (for saving)
for index, row in df1.iterrows():
  df1.loc[index,'symptoms_embeddings'] = df1.loc[index,'symptoms_embeddings'].tolist()
df1.to_csv('./database/disease_to_acu_and_symptom_modify_ZH_faceonly_embedding1.csv',index=False)
df1

,disease_ID,disease_name,ICD9,meridian,symptoms_origin,symptoms,acupoints,symptoms_embeddings
0,DIS0155,眼球震顫,379.5,足太陽膀胱經,['眼球震顫'],[眼球震顫],"[睛明, 球後]","[[-0.023460907861590385, 0.16163721680641174, ..."
1,DIS0417,新生兒窒息,768,督脈,"['皮膚青紫', '呼吸減慢', '心率減慢', '四肢略屈曲', '四肢癱軟', '對刺激...","[皮膚青紫, 呼吸減慢, 心率減慢, 四肢略屈曲, 四肢癱軟, 反應遲鈍]",[水溝],"[[0.4189542531967163, 0.67438143491745, -0.080..."
2,DIS0206,急性鼻竇炎,461.9,手陽明大腸經,['發燒、流鼻水、咳嗽、頭痛、咽痛，癥狀一直持續下去，鼻水變成化膿性的黃鼻涕，夜間咳嗽變得比...,"[發燒, 流鼻水, 咳嗽, 頭痛, 咽痛, 膿鼻涕, 夜間咳嗽, 臉痛]","[迎香, 印堂, 上迎香]","[[-0.3397959768772125, 0.48704326152801514, -0..."
3,DIS0128,假性近視,367,董氏奇穴七七部為小腿部位,['看遠看不清楚'],"[看遠看不清楚, 近視]","[睛明, 承泣]","[[0.11008584499359131, 0.16105933487415314, -0..."
4,DIS0419,新生兒痙攣,779,督脈,"['點頭單個發作時間短，發作一次為2～10秒，不易察覺；', '全身尤其是頭部和上半身向前屈...","[頻繁點頭, 意識喪失, 出汗, 面部青紫, 嗜睡]",[水溝],"[[-0.07805051654577255, 0.6575218439102173, 0...."
...,...,...,...,...,...,...,...,...
101,DIS0136,角膜結膜炎,370.4,足太陽膀胱經,"['疼痛、羞明、流淚和眼瞼痙攣', '睫狀充血、虹膜充血', '球結膜甚至眼瞼水腫', '視...","[疼痛, 羞明, 流淚, 眼瞼痙攣, 睫狀充血, 虹膜充血, 球結膜, 眼瞼水腫, 視力降低]","[太陽, 睛明]","[[0.19315718114376068, 0.572830855846405, -0.9..."
102,DIS0630,抑鬱癥,NaN,足太陽膀胱經,"['情緒低落', '興趣減退或快感消失', '焦慮', '思維遲緩、決斷能力下降、交流出現障...","[情緒低落, 興趣減退, 快感消失, 焦慮, 思維遲緩, 交流障礙, 記憶力下降, 漠不關心...","[太陽, 水溝, 印堂]","[[0.5364087224006653, 0.1496274322271347, 0.12..."
103,SYS0003,耳鳴,388.3,足太陽膀胱經,['耳鳴'],[耳鳴],"[攢竹, 迎香, 印堂]","[[0.1885010302066803, 0.40210625529289246, -1...."
104,SYS0023,失眠,780.5,足太陽膀胱經,['失眠'],[失眠],"[承漿, 攢竹, 太陽, 水溝, 印堂]","[[0.5523114204406738, 0.21947865188121796, -0...."


In [ ]:
queries = ['呼吸不順']
query_embeddings = embedder.encode(queries)

closest_n = 5
for query, query_embedding in zip(queries, query_embeddings):
  score = []
  for index, row in df.iterrows():
    dist = scipy.spatial.distance.cdist([query_embedding], df.loc[index,'symptoms_embeddings'], "cosine")[0]
    # take mean value of all diatence as score
    score.append(sum(dist) / len(dist))
  # sort by distence
  results = zip(range(len(score)), score)
  results = sorted(results, key=lambda x: x[1])
  print("======================")
  print("Query:", query)
  print("Result:Top 5 most similar sentences in corpus:")
  for idx, distance in results[0:closest_n]:
    print(df.iloc[idx,1], "(Score: %.4f)" % (1-distance))

Query: 呼吸不順
Result:Top 5 most similar sentences in corpus:
呼吸困難 (Score: 0.9034)
呼吸短促 (Score: 0.8559)
吞嚥困難 (Score: 0.8124)
排尿困難 (Score: 0.7890)
重聽 (Score: 0.7870)
